In [1]:
import torch
from pathlib import Path
from tqdm import tqdm

In [2]:
from pytorch_transformers import BertTokenizer, BertForMaskedLM

In [3]:
job_id="1212693"

In [ ]:
bi_encoder_inference = torch.load(f"output/melart_blink/{job_id}/top30_candidates/test.t7")
bi_encoder_inference.keys()

In [ ]:
len(bi_encoder_inference["indices"][0])

In [ ]:
max(bi_encoder_inference["labels"].flatten())

In [7]:
tokenizer=BertTokenizer.from_pretrained("bert-large-uncased")

In [ ]:
tokenizer.decode(bi_encoder_inference["candidate_vecs"][0][0].tolist(), skip_special_tokens=True)

In [ ]:
for i in range(10):
    print(tokenizer.decode(bi_encoder_inference["candidate_vecs"][0][i].tolist(), skip_special_tokens=True))

In [10]:
candidates_txt=open("/hpi/fs00/home/alejandro.sierra/MELArt_experiments/BLINK-main/data/melart_blink/documents/documents.jsonl").read().split("\n")

In [11]:
import json

In [ ]:
decoded_results=[]
for i,context in tqdm(enumerate(bi_encoder_inference["context_vecs"]),total=len(bi_encoder_inference["context_vecs"]),desc="Decoding"):
    sentence=tokenizer.decode(context.tolist(), skip_special_tokens=True)
    entities_ranked=[]
    for j in range(30):
        entities_ranked.append(tokenizer.decode(bi_encoder_inference["candidate_vecs"][i][j].tolist(), skip_special_tokens=True))
    indices=bi_encoder_inference["indices"][i]
    results=[]
    for j,ent in enumerate(entities_ranked):
        candidate=candidates_txt[indices[j]]
        candidate_obj=json.loads(candidate)
        results.append({"entity":ent,"index":indices[j], "document_id":candidate_obj["document_id"]})
    res={"sentence":sentence,"entities_ranked":results, "gt_rank":bi_encoder_inference["labels"][i].item()}
    decoded_results.append(res)

In [13]:
with open(f"output/melart_blink/{job_id}/top30_candidates/test.json","w") as f:
    json.dump(decoded_results,f,indent=4)